# Continuo MOdularizacion

Antes de continuar, darle alguna funcionalidad el endpoint de la raiz: `/` para que muestre la version, el nombre de la app, y el nombre de la base datos y el puerto.

### Paso 3 Crear Routers de API en `my-web-api/api/v1/endpoints/`

Aquí vamos a separar tus endpoints en archivos dedicados usando `APIRouter`

#### APIRouter

`APIRouter` es una clase fundamental en FastAPI que te permite organizar tu API en módulos o "sub-aplicaciones"

`APIRouter` es una instancia que actúa como un "mini-FastAPI" dentro de tu aplicación principal. Puedes definir rutas (`@router.get()`, `@router.post()`, etc.), dependencias (`Depends()`),

#### Modularizando las rutas `/` y `/cat`

- Crear el archivo `app/api/v1/endpoints/root.py`
- Este archivo contendrá los endpoints `/` y `/cat`

#### Modularizando la ruta `/students`

- Crear el archivo `app/api/v1/endpoints/root.py`
- Este archivo contendrá los endpoints `/students`

- Modifica `app/main.py` para incluir los nuevos routers:

```py
# Importar los routers de los endpoints
# Se ha corregido la importación: de 'app.api.v1.endpoints' a 'api.v1.endpoints'
from api.v1.endpoints import root, students, clients
...
...
# Incluir los routers en la aplicación principal(main.py)
app.include_router(root.router)
app.include_router(students.router)
```
Para probar:

1. Abre tu navegador y ve a `http://127.0.0.1:8000/docs`

2. Deberías ver la documentación de la API con los endpoints agrupados por tags (`root`, `student`, `clients`).

3.  Prueba cada uno de los endpoints:
    * `http://127.0.0.1:8000/`
    * `http://127.0.0.1:8000/cat`
    * `http://127.0.0.1:8000/student/1` (o cualquier ID de tu `datos.py`)
    * `http://127.0.0.1:8000/client/1` (o cualquier ID que tengas en tu DB MongoDB)

## Ejercicio

Como ejercicio final, termina de modularizar la ruta `clients`:

`api/v1/endpoints`

```py
from fastapi import APIRouter, HTTPException, Path
from fastapi.responses import JSONResponse
from core.database import db_client


router = APIRouter(tags=['clients'])


@router.get("/client/{client_id_to_find}")
async def get_client_by_id(client_id_to_find:int = Path(...,
                                                        gt=0,
                                                        description="ID del cliente a buscar")):
    """
    Busca un cliente por su ID en la base de datos MongoDB.

    Args:
        client_id_to_find (int): El ID del cliente a buscar. 

    Returns:
        JSONResponse: Un JSON con los datos del cliente si es encontrado.

    Raises:
        HTTPException:
            - 500 Internal Server Error si la conexión a la base de datos no está establecida.
            - 404 Not Found si el cliente con el ID especificado no existe.
    """
    if not db_client['client']:
        raise HTTPException(status_code=500, detail={"message": "la conexion a la BD ha fallado"})
    
    collection = db_client['client']['db-tienda'].clients 
    found_client = await collection.find_one({'id': client_id_to_find})

    if not found_client:
        message = f'cliente con id:{client_id_to_find} no encontrado'
        raise HTTPException(status_code=404, detail={"message":message})
    
    found_client['_id'] = str(found_client['_id'])
    return JSONResponse(status_code=200, content=found_client)
```